# Example 5: Stickleback morphometrics - body and armor-plate shape

Variation in continuous phenotypic traits like shape or area of certain structures are difficult to quantify with landmarks, because they are too complex or have no underlying assumption of homology. In this example, the number and area of armor plating was measured as a continuous trait in a two-step process: first, a mask was set around the posterior region that contains the plates, second, the red channel (highest signal-to-noise-ratio) of the image was thresholded. 

In this example we use the `watershed` algorithm, which helps to separate detected objects into "peaks of wanted information" and "valleys of unwanted information". The principle is explained here: https://docs.opencv.org/master/d3/db4/tutorial_py_watershed.html 

<div class="row; text-align: left">
    
<div class="col-md-6">
    
![Before](_figures/ex5_before.jpg)
    
**Input** - Stained threespine stickleback. The size of plates at a given plate index varies within at between ecotypes (e.g. lake and stream morphs)  
</div>
<div class="col-md-6">

![After](_figures/ex5_after.jpg)
    
**Results** - After applying adaptive thresholding and watershed algorithms, the plates separate. Where this is not the case they can be separated manually.
</div>
</div>